# Analysis - 2020
Given Uber ride information and COVID information from NYC, what what is the likelihood

In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('/home/felipe/repos/tcc/nyc_data/csv/2020_preprocessed_sample.csv')

In [2]:
y = dataset['SR_Flag']
X = dataset.drop('SR_Flag', axis=1)

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
trip_max_idx = X['trip_duration'].idxmax()

In [4]:
X.drop(X.index[trip_max_idx], 0, inplace=True)
y.drop(y.index[trip_max_idx], 0, inplace=True)

/home/felipe/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/home/felipe/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of Series.drop except for the argument 'labels' will be keyword-only
  


In [5]:
y = y.apply(lambda x: 0 if x == 0 else 1)

In [6]:
#      trip_duration <= 5
#  5 < trip_duration <= 10
# 10 < trip_duration <= 15
# 15 < trip_duration <= 20
# 20 < trip_duration <= 30
# 30 < trip_duration <= 45
# 45 < trip_duration <= 60
# 60 < trip_duration
X['trip_duration'] = pd.qcut(X['trip_duration'], 8, labels=False)

In [7]:
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, make_scorer

In [8]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_clf = DecisionTreeClassifier()

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, zero_division=0, average='weighted')
recall_scorer = make_scorer(recall_score)
f1_scorer = make_scorer(f1_score)
rmse_scorer = make_scorer(mean_squared_error, squared=False)

k_folds = range(2,12)

decision_tree_scores_cv = cross_validate(decision_tree_clf, X, y, cv=10, scoring={'accuracy': accuracy_scorer,
                                                                                  'precision': precision_scorer,
                                                                                  'recall': recall_scorer,
                                                                                  'f1': f1_scorer,
                                                                                  'rmse': rmse_scorer})
decision_tree_accuracy_scores = decision_tree_scores_cv['test_accuracy'].tolist()
decision_tree_precision_scores = decision_tree_scores_cv['test_precision'].tolist()
decision_tree_recall_scores = decision_tree_scores_cv['test_recall'].tolist()
decision_tree_f1_scores = decision_tree_scores_cv['test_f1'].tolist()
decision_tree_rmse_scores = decision_tree_scores_cv['test_rmse'].tolist()

decision_tree_scores = {'accuracy': decision_tree_accuracy_scores,
                        'precision': decision_tree_precision_scores,
                        'recall': decision_tree_recall_scores,
                        'f1': decision_tree_f1_scores,
                        'rmse': decision_tree_rmse_scores}

In [9]:
from sklearn.ensemble import RandomForestClassifier

depths = range(1,11)
random_forest_scores = []

for depth in depths:
    accuracy_scorer = make_scorer(accuracy_score)
    precision_scorer = make_scorer(precision_score, zero_division=0, average='weighted')
    recall_scorer = make_scorer(recall_score)
    f1_scorer = make_scorer(f1_score)
    rmse_scorer = make_scorer(mean_squared_error, squared=False)
    
    random_forest_clf = RandomForestClassifier(max_depth=depth, bootstrap=False)
    scores = cross_validate(random_forest_clf, X, y, cv=10, scoring={'accuracy': accuracy_scorer,
                                                                     'precision': precision_scorer,
                                                                     'recall': recall_scorer,
                                                                     'f1': f1_scorer,
                                                                     'rmse': rmse_scorer})
    random_forest_acc = scores['test_accuracy'].tolist()
    random_forest_prec = scores['test_precision'].tolist()
    random_forest_recall = scores['test_recall'].tolist()
    random_forest_f1 = scores['test_f1'].tolist()
    random_forest_rmse = scores['test_rmse'].tolist()
    
    random_forest_scores.append({'depth': depth,
                                 'accuracy': random_forest_acc,
                                 'precision': random_forest_prec,
                                 'recall': random_forest_recall,
                                 'f1': random_forest_f1,
                                 'rmse': random_forest_rmse})

In [10]:
from sklearn.neighbors import KNeighborsClassifier

k_values = range(1,6)
knn_scores = []

for k in k_values:
    accuracy_scorer = make_scorer(accuracy_score)
    precision_scorer = make_scorer(precision_score, zero_division=0, average='weighted')
    recall_scorer = make_scorer(recall_score)
    f1_scorer = make_scorer(f1_score)
    rmse_scorer = make_scorer(mean_squared_error, squared=False)

    knn_clf = KNeighborsClassifier(n_neighbors=k)
    scores = cross_validate(knn_clf, X, y, cv=10, scoring={'accuracy': accuracy_scorer,
                                                       'precision': precision_scorer,
                                                       'recall': recall_scorer,
                                                       'f1': f1_scorer,
                                                       'rmse': rmse_scorer})
    knn_acc = scores['test_accuracy'].tolist()
    knn_prec = scores['test_precision'].tolist()
    knn_recall = scores['test_recall'].tolist()
    knn_f1 = scores['test_f1'].tolist()
    knn_rmse = scores['test_rmse'].tolist()
    
    knn_scores.append({'k': k,
                       'accuracy': knn_acc,
                       'precision': knn_prec,
                       'recall': knn_recall,
                       'f1': knn_f1,
                       'rmse': knn_rmse})

In [11]:
from sklearn.cluster import KMeans

k_values = range(1,6)
kmeans_scores = []

for k in k_values:
    accuracy_scorer = make_scorer(accuracy_score)
    precision_scorer = make_scorer(precision_score, zero_division=0, average='weighted')
    recall_scorer = make_scorer(recall_score)
    f1_scorer = make_scorer(f1_score)
    rmse_scorer = make_scorer(mean_squared_error, squared=False)
    kmeans_recall_scorer = make_scorer(recall_score, zero_division=1, average='weighted')
    kmeans_f1_scorer = make_scorer(f1_score, zero_division=1, average='weighted')
    
    kmeans = KMeans(n_clusters=k)
    acc_scores = cross_val_score(kmeans, X, y, cv=10, scoring='accuracy')
    prec_scores = cross_val_score(kmeans, X, y, cv=10, scoring=precision_scorer)
    recall_scores = cross_val_score(kmeans, X, y, cv=10, scoring=kmeans_recall_scorer)
    f1_scores = cross_val_score(kmeans, X, y, cv=10, scoring=kmeans_f1_scorer)
    rmse_scores = cross_val_score(kmeans, X, y, cv=10, scoring=rmse_scorer)
    
    kmeans_scores.append({'k': k,
                          'accuracy': acc_scores.tolist(),
                          'precision': prec_scores.tolist(),
                          'recall': recall_scores.tolist(),
                          'f1': f1_scores.tolist(),
                          'rmse': rmse_scores.tolist()})

In [12]:
from sklearn.neural_network import MLPClassifier

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, zero_division=0, average='weighted')
recall_scorer = make_scorer(recall_score)
f1_scorer = make_scorer(f1_score)
rmse_scorer = make_scorer(mean_squared_error, squared=False)

mlp_classifier = MLPClassifier(solver='adam', hidden_layer_sizes=(100, 5))
mlp_scores = cross_validate(mlp_classifier, X, y, cv=10, scoring={'accuracy': accuracy_scorer,
                                                                  'precision': precision_scorer,
                                                                  'recall': recall_scorer,
                                                                  'f1': f1_scorer,
                                                                  'rmse': rmse_scorer})
mlp_acc = mlp_scores['test_accuracy'].tolist()
mlp_prec = mlp_scores['test_precision'].tolist()
mlp_recall = mlp_scores['test_recall'].tolist()
mlp_f1 = mlp_scores['test_f1'].tolist()
mlp_rmse = mlp_scores['test_rmse'].tolist()

mlp_scores = {'accuracy': mlp_acc,
              'precision': mlp_prec,
              'recall': mlp_recall,
              'f1': mlp_f1,
              'rmse': mlp_rmse}

In [13]:
from sklearn.naive_bayes import GaussianNB

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, zero_division=0, average='weighted')
recall_scorer = make_scorer(recall_score)
f1_scorer = make_scorer(f1_score)
rmse_scorer = make_scorer(mean_squared_error, squared=False)

gnb_classifier = GaussianNB()
gnb_scores = cross_validate(gnb_classifier, X, y, cv=10, scoring={'accuracy': accuracy_scorer,
                                                                  'precision': precision_scorer,
                                                                  'recall': recall_scorer,
                                                                  'f1': f1_scorer,
                                                                  'rmse': rmse_scorer})
gnb_acc = gnb_scores['test_accuracy'].tolist()
gnb_prec = gnb_scores['test_precision'].tolist()
gnb_recall = gnb_scores['test_recall'].tolist()
gnb_f1 = gnb_scores['test_f1'].tolist()
gnb_rmse = gnb_scores['test_rmse'].tolist()

gnb_scores = {'accuracy': gnb_acc,
              'precision': gnb_prec,
              'recall': gnb_recall,
              'f1': gnb_f1,
              'rmse': gnb_rmse}

In [14]:
from sklearn.naive_bayes import ComplementNB

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, zero_division=0, average='weighted')
recall_scorer = make_scorer(recall_score)
f1_scorer = make_scorer(f1_score)
rmse_scorer = make_scorer(mean_squared_error, squared=False)

cnb_classifier = ComplementNB()
cnb_scores = cross_validate(cnb_classifier, X, y, cv=10, scoring={'accuracy': accuracy_scorer,
                                                                  'precision': precision_scorer,
                                                                  'recall': recall_scorer,
                                                                  'f1': f1_scorer,
                                                                  'rmse': rmse_scorer})
cnb_acc = cnb_scores['test_accuracy'].tolist()
cnb_prec = cnb_scores['test_precision'].tolist()
cnb_recall = cnb_scores['test_recall'].tolist()
cnb_f1 = cnb_scores['test_f1'].tolist()
cnb_rmse = cnb_scores['test_rmse'].tolist()

cnb_scores = {'accuracy': cnb_acc,
              'precision': cnb_prec,
              'recall': cnb_recall,
              'f1': cnb_f1,
              'rmse': cnb_rmse}

In [15]:
from sklearn.naive_bayes import MultinomialNB

accuracy_scorer = make_scorer(accuracy_score)
precision_scorer = make_scorer(precision_score, zero_division=0, average='weighted')
recall_scorer = make_scorer(recall_score)
f1_scorer = make_scorer(f1_score)
rmse_scorer = make_scorer(mean_squared_error, squared=False)

mnb_classifier = MultinomialNB()
mnb_scores = cross_validate(mnb_classifier, X, y, cv=10, scoring={'accuracy': accuracy_scorer,
                                                                  'precision': precision_scorer,
                                                                  'recall': recall_scorer,
                                                                  'f1': f1_scorer,
                                                                  'rmse': rmse_scorer})
mnb_acc = mnb_scores['test_accuracy'].tolist()
mnb_prec = mnb_scores['test_precision'].tolist()
mnb_recall = mnb_scores['test_recall'].tolist()
mnb_f1 = mnb_scores['test_f1'].tolist()
mnb_rmse = mnb_scores['test_rmse'].tolist()

mnb_scores = {'accuracy': mnb_acc,
              'precision': mnb_prec,
              'recall': mnb_recall,
              'f1': mnb_f1,
              'rmse': mnb_rmse}

In [16]:
supervised_scores = {'decision_tree': decision_tree_scores,
                     'random_forest': random_forest_scores,
                     'knn': knn_scores,
                     'kmeans': kmeans_scores,
                     'mlp': mlp_scores,
                     'gnb': gnb_scores,
                     'cnb': cnb_scores,
                     'mnb': mnb_scores}

In [17]:
import json

with open('2020_scores.json', 'w') as json_2020:
    json.dump(supervised_scores, json_2020, indent=4)